In [1]:
import numpy as np
import ipywidgets as widgets
import time

from ase.build import mx2

from abtem.structures import orthogonalize_cell
from abtem.waves import PlaneWave
from abtem.transfer import CTF
from abtem.visualize.interactive.canvas import Canvas
from abtem.visualize.interactive.artists import ImageArtist, ColorBar, LinesArtist, MeasurementArtist1d, MeasurementArtist2d
from abtem.visualize.interactive.tools import PanZoomTool, BoxZoomTool

# Demo of abTEM's interactive module

This is a short demonstration of abTEM's interactive module. The module is currently under development and everything is subject to change. 

Interactions in abTEM are based on `ipywidgets`, `bqplot` and `bqplot-image-gl`, all of these packages use the `traitlets` package for running 'on change' callbacks. The best way to learn about `bqplot` is the [examples](https://github.com/bqplot/bqplot/tree/master/examples) and you can learn about `trailets` through the [documentation](https://traitlets.readthedocs.io/en/stable/using_traitlets.html).

The three main types of objects implemented in abTEM's interactive module are the `Canvas`, `Artist` and `Tool` object. The goal of the module is simply to act as the glue between abTEM and bqplot to make interactive visualizations of TEM simulations easier.

## Basics

The `Canvas` is a base object on which visualizations can be placed.

In [2]:
canvas = Canvas()
canvas.figure

Figure(axes=[Axis(scale=LinearScale(allow_padding=False)), Axis(orientation='vertical', scale=LinearScale(allo…

An `Artist` is an object that takes some other object and visualize it on a `Canvas`. Below we create an `ImageArtist` which takes a 2d array and visualize it as an image. One `Canvas` can have multiple artists, hence the artists are given as a dictionary.

In [3]:
artist = ImageArtist()

array = np.random.rand(32,32)

artist.image = array

canvas.artists = {'my_image' : artist}

canvas.x_label = 'my_x_label'

The visualization can be updated by modifying a trait. Here, we simulate a periodic update of the image.

In [4]:
for i in range(50):
    time.sleep(.025) # some computation
    artist.image = np.random.rand(32,32)

The artists have some builtin widgets for making common adjustments. 

In [5]:
artist.color_scheme_picker

Dropdown(description='Scheme', options=('Greys', 'viridis', 'inferno', 'plasma', 'magma', 'Spectral', 'RdBu'),…

The same can be accomplished by setting the scheme manually.

In [6]:
artist.color_scheme = 'plasma'

A `Tool` can modify the traits of an `Artist` or the `Canvas`. Here we create the basic navigation tools. Picking a tool will activate it on the `Canvas` above. 

In [7]:
canvas.tools = {'Pan':PanZoomTool(), 'Zoom':BoxZoomTool()}

canvas.toolbar

Custom interactions can be created using callback functions (scroll up to see the effects).

In [8]:
array = np.random.rand(32,32) * 2

def update(change):
    artist.image = array ** change['new']

slider = widgets.FloatSlider(min=.1, max=10, step=.1)

slider.observe(update, 'value')

slider

FloatSlider(value=0.1, max=10.0, min=0.1)

Everything can be composed using ipywidgets.

In [9]:
canvas = Canvas()

artist = ImageArtist()
artist.image = np.random.rand(32,32)
canvas.artists = {'my_image' : artist}

canvas.tools = {'Pan':PanZoomTool(), 'Zoom':BoxZoomTool()}

widgets.HBox([widgets.VBox([canvas.figure, artist.colorbar]), 
              widgets.VBox([artist.color_scheme_picker, canvas.toolbar, slider])])

To create a 1d visualization we may want to modify the `Canvas`. The `lock_scale` trait ensures equal x and y-scale and thus square pixels, here we disable it.

In [10]:
canvas = Canvas(lock_scale=False, width=600, height=300)

canvas.figure

Figure(axes=[Axis(scale=LinearScale(allow_padding=False)), Axis(orientation='vertical', scale=LinearScale(allo…

Multiple artists are added to the same `Canvas`.

In [11]:
linesartist1 = LinesArtist()
linesartist1.x = np.linspace(0,10,100)
linesartist1.y = np.random.rand(100)

linesartist2 = LinesArtist(colors='blue')
linesartist2.x = np.linspace(0,20,100)
linesartist2.y = np.random.rand(100)

canvas.artists = {'line1' : linesartist1, 'line2' : linesartist2}

The visibility of an `Artist` can be modified using the widget below.

In [12]:
canvas.visibility_checkboxes

## Example: HRTEM with CTF

The next cells show how to put together a real interactive example. The result is similar to calling the `.apply` method of the `CTF` object with `interact=True`.

First, standard abTEM code is run to obtain an exit wavefunction for the visualization.

In [13]:
atoms = mx2(formula='MoS2', kind='2H', a=3.18, thickness=3.19, size=(1, 1, 1), vacuum=None)
atoms = orthogonalize_cell(atoms) * (3, 2, 1)
atoms.center(vacuum=2, axis=2)

exit_wave = PlaneWave(energy=300e3, sampling=.05).multislice(atoms,pbar=False)
ctf = CTF(energy=300e3, rolloff=.05)

Two Canvas objects are created for displaying the HRTEM image and a radial 1d slice of the CTF. We create a `MeasurementArtist2d` and two `MeasurementArtist1d`, these artists takes an abTEM `Measurement` object and creates a visualization.

In [14]:
canvas1 = Canvas()
image_artist = MeasurementArtist2d()
canvas1.artists = {'image' : image_artist}

canvas2 = Canvas(lock_scale=False)
ctf_artist = MeasurementArtist1d()
envelope_artist = MeasurementArtist1d()
canvas2.artists = {'ctf' : ctf_artist, 'envelope':envelope_artist}
canvas2.y_scale.min = -1 
canvas2.y_scale.max = 1 

In [15]:
# Create widgets
defocus_slider = widgets.FloatSlider(description='defocus', min=-100, max=100, step=1)
Cs_slider = widgets.FloatSlider(description='Cs', min=-1e5, max=1e5, step=1)
aperture_slider = widgets.FloatSlider(description='aperture', min=5, max=100, value=20, step=1)

# Create callback
def update(*args):
    # Fetch values from sliders
    ctf.defocus = defocus_slider.value
    ctf.Cs = Cs_slider.value
    ctf.semiangle_cutoff = aperture_slider.value
    
    # abTEM calculations
    image = ctf.apply(exit_wave).intensity()[0]
    ctf_profiles = ctf.profiles(max_semiangle=100)
    
    # Set the objects to visualize
    ctf_artist.measurement = ctf_profiles['ctf']
    envelope_artist.measurement = ctf_profiles['envelope']
    image_artist.measurement = image
    
# Attach callback
defocus_slider.observe(update, 'value')
Cs_slider.observe(update, 'value')
aperture_slider.observe(update, 'value')

# Run callback once to refresh, and adjust canvas to match the artists
update()
canvas1.adjust_limits_to_artists()
canvas1.adjust_labels_to_artists()
canvas2.adjust_limits_to_artists(adjust_y=False) # we manually fixed the y-limits above
canvas2.adjust_labels_to_artists()

In [16]:
widgets.VBox([widgets.HBox([canvas1.figure, canvas2.figure]), 
              widgets.VBox([defocus_slider, Cs_slider, aperture_slider])])